In [1]:
#For model data
import sys
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt


#for calculating time difference
from datetime import datetime, timedelta


#Tools for evaraging model data
sys.path.append(r'./tools')

In [66]:
dataset0 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/fou-hi/barents_eps_eps/barents_eps_20221118T12Z.nc")
dataset1 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/fou-hi/barents_eps_eps/barents_eps_20221118T18Z.nc")
dataset2 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/fou-hi/barents_eps_eps/barents_eps_20221119T00Z.nc")
dataset3 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/fou-hi/barents_eps_eps/barents_eps_20221119T06Z.nc")
dataset4 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/fou-hi/barents_eps_eps/barents_eps_20221119T12Z.nc")
dataset5 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/fou-hi/barents_eps_eps/barents_eps_20221119T18Z.nc")
dataset6 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/fou-hi/barents_eps_eps/barents_eps_20221120T00Z.nc")
dataset7 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/fou-hi/barents_eps_eps/barents_eps_20221120T06Z.nc")

In [68]:
datasets = [dataset0, dataset1, dataset2, dataset3, dataset4, dataset5, dataset6,dataset7]

In [74]:
dataset0.ensemble_member

<xarray.DataArray 'ensemble_member' (ensemble_member: 6)>
array([12, 13, 14, 15, 16, 17], dtype=int32)
Coordinates:
  * ensemble_member  (ensemble_member) int32 12 13 14 15 16 17
Attributes:
    long_name:      ensemble run member
    standard_name:  realization

In [86]:
print("Ensembles")
for idx, ensemble in enumerate(datasets):
    print(f"dataset{idx}: {datasets[idx].ensemble_member.values}")
print("Start time")
for idx, ensemble in enumerate(datasets):
    print(f"dataset{idx}: {datasets[idx].time[0].values} - {datasets[idx].time[-1].values}, {len(datasets[idx].time)- 1} forecast")

Ensembles
dataset0: [12 13 14 15 16 17]
dataset1: [18 19 20 21 22 23]
dataset2: [0 1 2 3 4 5]
dataset3: [ 6  7  8  9 10 11]
dataset4: [12 13 14 15 16 17]
dataset5: [18 19 20 21 22 23]
dataset6: [0 1 2 3 4 5]
dataset7: [ 6  7  8  9 10 11]
Start time
dataset0: 2022-11-18T12:00:00.000000000 - 2022-11-21T06:00:00.000000000, 66 forecast
dataset1: 2022-11-18T18:00:00.000000000 - 2022-11-21T12:00:00.000000000, 66 forecast
dataset2: 2022-11-19T00:00:00.000000000 - 2022-11-21T18:00:00.000000000, 66 forecast
dataset3: 2022-11-19T06:00:00.000000000 - 2022-11-22T00:00:00.000000000, 66 forecast
dataset4: 2022-11-19T12:00:00.000000000 - 2022-11-22T06:00:00.000000000, 66 forecast
dataset5: 2022-11-19T18:00:00.000000000 - 2022-11-22T12:00:00.000000000, 66 forecast
dataset6: 2022-11-20T00:00:00.000000000 - 2022-11-22T18:00:00.000000000, 66 forecast
dataset7: 2022-11-20T06:00:00.000000000 - 2022-11-23T00:00:00.000000000, 66 forecast


In [89]:
# Generate fixed variables

X = dataset0['X']
Y = dataset0['Y']
lon = dataset0['lon']
lat = dataset0['lat']
model_proj4 = dataset0.projection_lambert.proj4

In [95]:
#Import variables for extracting subsets from SAR1-SAR2_drift.ipynb
%store -r r
%store -r c
%store -r X_subset
%store -r Y_subset
%store -r lon_subset
%store -r lat_subset

In [ ]:
# Generates lists of variables which changes from dataset to dataset
ensemble_members = []
ice_u = []
ice_v = []
siconc = []

for idx, ds in enumerate(datasets):
    ensemble_member = ds['ensemble_member']
    ensemble_members.append(ensemble_member)
    
    ice_u = ds['ice_u']
    ice_u.append(ice_u)
    
    ice_v = ds['ice_v']
    ice_v.append(ice_v)
    
    siconc = ds['siconc']
    siconc.append(siconc)

In [ ]:
# Sea ice related variables
ensemble_member = data['ensemble_member']
ice_u = data['ice_u'] # ice velocity (x) 
ice_v = data['ice_v'] # ice velocity (y)
siconc = data['ice_concentration'] # ice area  (aggregate)

# Other variables
u = data['u'] # sea water x velocity
v = data['v'] # sea water y velocity
Uwind = data['Uwind'] # surface u-wind component
Vwind = data['Vwind'] #surface v-wind component


print(f'Model projection is {data.projection_lambert.grid_mapping_name} \nwith the following parameters in proj4 format: {model_proj4}')

In [ ]:
#Make list of all variables

In [44]:
from model_data_prep import round_start_time, round_end_time, time_difference

# SAR images timestamps
t_sar1 = '2022-11-19T07:22:08'
t_sar2 = '2022-11-20T08:02:59'

# Rounding the SAR timestamps to align with the nearest whole hour of model timestamps
t_start = round_start_time(t_sar1)
t_end = round_end_time(t_sar2)

print(f'SAR1 time is {t_sar1}, Model start time for the time period is {t_start}')
print(f'SAR2 time is {t_sar2}, Model end time for the time period is {t_end}')


# Extracting the model time variavle data corresponding to the time period between the rounded start and end times
time_period = dataset0.time.sel(time=slice(t_start,t_end))

print(f'The duration of the time period is {len(time_period)-1} whole hours')

SAR1 time is 2022-11-19T07:22:08, Model start time for the time period is 2022-11-19T07:00:00
SAR2 time is 2022-11-20T08:02:59, Model end time for the time period is 2022-11-20T09:00:00
The duration of the time period is 26 whole hours


In [41]:
time_period

<xarray.DataArray 'time' (time: 27)>
array(['2022-11-19T07:00:00.000000000', '2022-11-19T08:00:00.000000000',
       '2022-11-19T09:00:00.000000000', '2022-11-19T10:00:00.000000000',
       '2022-11-19T11:00:00.000000000', '2022-11-19T12:00:00.000000000',
       '2022-11-19T13:00:00.000000000', '2022-11-19T14:00:00.000000000',
       '2022-11-19T15:00:00.000000000', '2022-11-19T16:00:00.000000000',
       '2022-11-19T17:00:00.000000000', '2022-11-19T18:00:00.000000000',
       '2022-11-19T19:00:00.000000000', '2022-11-19T20:00:00.000000000',
       '2022-11-19T21:00:00.000000000', '2022-11-19T22:00:00.000000000',
       '2022-11-19T23:00:00.000000000', '2022-11-20T00:00:00.000000000',
       '2022-11-20T01:00:00.000000000', '2022-11-20T02:00:00.000000000',
       '2022-11-20T03:00:00.000000000', '2022-11-20T04:00:00.000000000',
       '2022-11-20T05:00:00.000000000', '2022-11-20T06:00:00.000000000',
       '2022-11-20T07:00:00.000000000', '2022-11-20T08:00:00.000000000',
       '2022-11-20T09:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2022-11-19T07:00:00 ... 2022-11-20T09:00:00
Attributes:
    long_name:      time since initialization
    field:          time, scalar, series
    axis:           T
    standard_name:  time
    _ChunkSizes:    1

In [45]:
ensemble_member0 = dataset0['ensemble_member']
ensemble_member1 =  dataset1['ensemble_member']
ensemble_member2 =  dataset2['ensemble_member']
ensemble_member3 =  dataset3['ensemble_member']
ensemble_member4 =  dataset4['ensemble_member']

numpy.datetime64('2022-11-19T06:00:00.000000000')

In [65]:
print(f"Ensembles:\ndataset0: {ensemble_member0.values}\ndataset1: {ensemble_member1.values}\ndataset2: {ensemble_member2.values}\ndataset3: {ensemble_member3.values}\ndataset4: {ensemble_member4.values}")
print(f"Time start:\ndataset0: {dataset0.time[0].values}\ndataset1: {dataset1.time[0].values}\ndataset2: {dataset2.time[0].values}\ndataset3: {dataset3.time[0].values}\ndataset4: {dataset4.time[0].values}")

Ensembles:
dataset0: [ 6  7  8  9 10 11]
dataset1: [12 13 14 15 16 17]
dataset2: [18 19 20 21 22 23]
dataset3: [0 1 2 3 4 5]
dataset4: [ 6  7  8  9 10 11]
Time start:
dataset0: 2022-11-19T06:00:00.000000000
dataset1: 2022-11-19T12:00:00.000000000
dataset2: 2022-11-19T18:00:00.000000000
dataset3: 2022-11-20T00:00:00.000000000
dataset4: 2022-11-20T06:00:00.000000000


In [ ]:
meanfor t in range(1,len(time_period)):
    
    #extracting mean velocity data using polygon mask for area of interest
    u1 = ice_u.sel(time=time_period[t-1]).mean(dim='ensemble_member')
    u2 = ice_u.sel(time=time_period[t]).mean(dim='ensemble_member')
    
    
    v1 = ice_v.sel(time=time_period[t-1]).mean(dim='ensemble_member')
    v2 = ice_v.sel(time=time_period[t]).mean(dim='ensemble_member')

In [ ]:
for t in range(1, len(time_period)):
    print(f'{t} hour done')    
    # Extract instantaneous velocity data for the hour start and end using polygon mask for area of interest
    u1 = ice_u.sel(time=time_period[t-1])#.where(finalMask>0)
    u2 = ice_u.sel(time=time_period[t])#.where(finalMask>0)

    v1 = ice_v.sel(time=time_period[t-1])#.where(finalMask>0)
    v2 = ice_v.sel(time=time_period[t])#.where(finalMask>0)

    # Calculate displacement along lon (u) and lat (v) based on mean velovity
    # The first and the last displacemnets are calculated differently based on time difference with the SAR acquisitions
    if t == 1:
        u_displacement = (u2+u1)*(3600-time_diff_start)/2
        v_displacement = (v2+v1)*(3600-time_diff_start)/2
        #print("start", t, u_displacement.values)
    elif t in range(2, len(time_period)-1):
        u_displacement = (u2+u1)*3600/2
        v_displacement = (v2+v1)*3600/2
        #print(t, u_displacement.values)
    elif t == len(time_period)-1:
        u_displacement = (u2+u1)*(3600-time_diff_end)/2
        v_displacement = (v2+v1)*(3600-time_diff_end)/2